In [ ]:
!pip install python-bcb
!pip install dash

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

from bcb import sgs

#Selecionar os Dados
IPCA = sgs.get({'IPCA':433}, start='2000-01-01').reset_index()
IPCAab = sgs.get({'IPCAab':1635}, start='2000-01-01').reset_index()
IPAM = sgs.get({'IPAM':7450}, start='2000-01-01').reset_index()
Ccb = sgs.get({'Custo Cesta Basica - SP':7493}, start='2000-01-01').reset_index()

#Limpar/Uniformizar os Dados
def datas(df, nome_coluna):
  df.columns = ['Data', nome_coluna]
  df['Data'] = pd.to_datetime(df['Data'])
  return df

dfipca = datas(IPCA, 'IPCA')
dfipcaab = datas(IPCAab, 'IPCA Alimentacao e Bebidas')
dfipam = datas(IPAM, 'IPA-M')
dfccb = datas(Ccb, 'Custo Cesta Basica - SP')

#Derivar os Dados
def variacao(df, nome_coluna, variacao):
  df[variacao] = df[nome_coluna].pct_change() *100
  return df

dfipca = variacao(dfipca, 'IPCA', 'Variacao IPCA')
dfipcaab = variacao(dfipcaab, 'IPCA Alimentacao e Bebidas', 'Variacao IPCA-AB')
dfipam = variacao(dfipam, 'IPA-M', 'Variacao IPA-M')
dfccb = variacao(dfccb, 'Custo Cesta Basica - SP', 'Variacao Custo Cesta Basica - SP')

#Integrar os Dados
dfintegrar = dfipca.merge(dfipcaab, on='Data', how='inner')
dfintegrar = dfintegrar.merge(dfipam, on='Data', how='inner')
dfintegrar = dfintegrar.merge(dfccb, on='Data', how='inner')

#Formatar os Dados
dfintegrar = dfintegrar.round(2)
dfipca = dfipca.round(2)
dfipcaab = dfipcaab.round(2)
dfipam = dfipam.round(2)
dfccb = dfccb.round(2)

In [6]:
from dash import dcc
import dash
from dash import html
from dash import Input, Output
import plotly.express as px
import plotly.graph_objects as go

df = dfintegrar

app = dash.Dash(__name__)

app.layout = html.Div([

    html.Div([
      html.Div([
        dcc.Checklist(
          id = 'bcb',
          options=[{'label': 'IPCA', 'value': 'IPCA'},
                  {'label': 'IPCA - Alimentacao e Bebidas', 'value': 'IPCA Alimentacao e Bebidas'},
                  {'label': 'Indice de Preços por Atacado-Mercado', 'value': 'IPA-M'},
                  {'label': 'Custo Cesta Basica em São Paulo', 'value': 'Custo Cesta Basica - SP'}
                  ], value=['IPCA']
        )
      ], style={'width': '30%', 'marginTop': '10px'}),

      html.Div([

        html.Label("Tipo de grafico:"),

        dcc.RadioItems(
          id = 'grafico',
          options=[{'label': 'Grafico de Linhas', 'value': 'line'},
                  {'label': 'Grafico de Barras', 'value': 'bar'}
                  ], value='line'
        )
      ], style={'width': '30%', 'marginTop': '10px'} ),

      html.Div([
        dcc.RangeSlider(
          id = 'anos',
          min = df['Data'].dt.year.min(),
          max = df['Data'].dt.year.max(),
          step = 1,
          value=[df['Data'].dt.year.min(), df['Data'].dt.year.max()],
          marks={str(ano): str(ano) for ano in df['Data'].dt.year.unique()}
        )

      ], style={'width': '100%', 'marginTop': '30px'})

    ], style={'display': 'flex', 'gap': '10px', 'marginBottom': '20px'}),

    dcc.Graph(id = 'graph'),

    dcc.RadioItems(
      id = 'bcbcorrelacao',
      options=[{'label': 'IPCA', 'value': 'IPCA'},
              {'label': 'IPCA - Alimentacao e Bebidas', 'value': 'IPCA Alimentacao e Bebidas'},
              {'label': 'Indice de Preços por Atacado Mercado', 'value': 'IPA-M'}
              ], value='IPCA'
    ),

    dcc.Graph(id = 'graphdois'),

], style={'backgroundColor': '#ffffff'})

@app.callback(
  Output('graph', 'figure'), Input('grafico', 'value'), Input('bcb', 'value'), Input('anos', 'value')
)

def update_graph(tipo_grafico, opcoes_selecionadas, data_escolhida):
  fig = go.Figure()

  df_data = df[df['Data'].dt.year.between(data_escolhida[0], data_escolhida[1])]

  for op in opcoes_selecionadas:
      if op == "Custo Cesta Basica - SP":
        if tipo_grafico == 'line':
          fig.add_trace(go.Scatter(x=df_data["Data"], y=df_data[op], name=op, yaxis="y2", mode='lines'))
        elif tipo_grafico == 'bar':
          fig.add_trace(go.Bar(x=df_data["Data"], y=df_data[op], name=op, yaxis="y2", opacity=0.5))

      else:
        if tipo_grafico == 'line':
          fig.add_trace(go.Scatter(x=df_data["Data"], y=df_data[op], name=op, yaxis="y", mode='lines'))
        elif tipo_grafico == 'bar':
          fig.add_trace(go.Bar(x=df_data["Data"], y=df_data[op], name=op, yaxis="y"))

  fig.update_layout(xaxis=dict(title="Data"), yaxis=dict(title="Índices"),
                    yaxis2=dict(title="Cesta Básica (R$)", overlaying="y", side="right", showgrid=False),
                    title = "Evolução Temporal dos Índices Econômicos",
                    legend=dict(orientation="h", yanchor="bottom", y=-0.3, xanchor="center", x=0.5, bordercolor = "black", borderwidth=1),
                    barmode="group" if tipo_grafico == "bar" else None
  )

  fig.update_xaxes(dtick="M12" )
  return fig

@app.callback(
  Output('graphdois', 'figure'), [Input('bcbcorrelacao', 'value'), Input('anos', 'value')]
)

def update_graph2(opcoes_correlacao,anos):
  z = "Custo Cesta Basica - SP"
  y = opcoes_correlacao
  i = f"{y}"

  dfcorrelacao = df.copy()
  dfcorrelacao[y] = df[y] * 1000
  dfcorrelacao.rename(columns={y : i}, inplace=True)

  fig = px.scatter(dfcorrelacao, x="Data", y=[i,z],
                    trendline = "ols", title = f"Dispersão e Correlação: {z} e {i}", opacity = 0.5)

  fig.update_layout(legend=dict(orientation="h", yanchor="top", y=1, xanchor="center", x=0.5, bordercolor = "black", borderwidth=1), legend_title = None)
  fig.update_yaxes(visible = False)
  fig.update_xaxes(dtick = "M12" )
  return fig


if __name__ == '__main__':
  app.run(debug=True)

<IPython.core.display.Javascript object>